# Análise exploratória de dados

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os

# Importando os dados

In [41]:
names = ['Sex', 'Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight', 'Rings']

In [42]:
# do diretorio da catherine
df = pd.read_csv(r"\Users\User\equipe-01\data\raw\abalone.csv", header=None, names=names)

In [43]:
df.head()

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


## 1. Descrição dos dados

## Dicionário

### Criando o dicionário

In [58]:
df_dict = {
    'sexo': {'descrição': ' sexo do molusco', 'tipo': ' qualitativo', 'subtipo': ' nominal'}, 
    'comprimento': {'descrição': ' tamanho máximo da concha (mm)', 'tipo': ' quantitativo', 'subtipo': ' contínuo'}, 
    'diâmetro': {'descrição':' medida perpendicular ao comprimento (mm)', 'tipo': ' qualitativo', 'subtipo': ' contínuo'}, 
    'altura': {'descrição':' altura total do molusco com casca (mm)', 'tipo': ' qualitativo', 'subtipo': ' contínuo'}, 
    'peso total': {'descrição':' peso total do molusco (g)', 'tipo': ' quantitativo', 'subtipo': ' contínuo'}, 
    'peso sem casca': {'descrição':' peso do molusco sem casca (g)', 'tipo': ' quantitativo', 'subtipo': ' contínuo'}, 
    'peso do intestino': {'descrição':' peso do intestino após sangrar (g)', 'tipo': ' quantitativo', 'subtipo': ' contínuo'},
    'peso da concha': {'descrição': ' peso da concha seca (g)', 'tipo': ' quantitativo', 'subtipo': ' contínuo'},
    'anéis': {'descrição': ' quantidade de anéis, +1.5 é a idade do abalone', 'tipo': ' quantitativo', 'subtipo': ' discreto' }
}

In [59]:
df_dict

{'sexo': {'descrição': ' sexo do molusco',
  'tipo': ' qualitativo',
  'subtipo': ' nominal'},
 'comprimento': {'descrição': ' tamanho máximo da concha (mm)',
  'tipo': ' quantitativo',
  'subtipo': ' contínuo'},
 'diâmetro': {'descrição': ' medida perpendicular ao comprimento (mm)',
  'tipo': ' qualitativo',
  'subtipo': ' contínuo'},
 'altura': {'descrição': ' altura total do molusco com casca (mm)',
  'tipo': ' qualitativo',
  'subtipo': ' contínuo'},
 'peso total': {'descrição': ' peso total do molusco (g)',
  'tipo': ' quantitativo',
  'subtipo': ' contínuo'},
 'peso sem casca': {'descrição': ' peso do molusco sem casca (g)',
  'tipo': ' quantitativo',
  'subtipo': ' contínuo'},
 'peso do intestino': {'descrição': ' peso do intestino após sangrar (g)',
  'tipo': ' quantitativo',
  'subtipo': ' contínuo'},
 'peso da concha': {'descrição': ' peso da concha seca (g)',
  'tipo': ' quantitativo',
  'subtipo': ' contínuo'},
 'anéis': {'descrição': ' quantidade de anéis, +1.5 é a idade d

### Salvando o dicionário como csv

In [60]:
directory = r'\Users\User\equipe-01\data\external'
csv_file = 'abalone_dict.csv'

#checando se o diretório existe
if not os.path.exists(directory):
    os.makedirs(directory)

csv_path = os.path.join(directory, csv_file)

with open(csv_path, 'w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(['Preditor', 'Descrição', 'Tipo', 'Subtipo'])
    for column, attributes in df_dict.items():
        csv_writer.writerow([column, attributes['descrição'], attributes['tipo'], attributes['subtipo']])

## 2. Perguntas de partida e hipóteses

## 3. Insights